In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import pandas as pd

# Set the path to the CSV file
csv_file = "/content/drive/MyDrive/OCT-AND-EYE-FUNDUS-DATASET/EYE FUNDUS.csv"

# Read the CSV file
data = pd.read_csv(csv_file)

# Modify the DR label
data['DR'] = data['DR'].replace(['PDR', 'NPDR', '-'], 1)

# Save the modified CSV file
data.to_csv("/content/drive/MyDrive/OCT-AND-EYE-FUNDUS-DATASET/fundus.csv", index=False)


In [14]:
import os
import shutil
import pandas as pd

# Set the paths
images_folder = "/content/drive/MyDrive/OCT-AND-EYE-FUNDUS-DATASET/EYE FUNDUS"
csv_file = "/content/drive/MyDrive/OCT-AND-EYE-FUNDUS-DATASET/fundus.csv"

# Read the CSV file
data = pd.read_csv(csv_file)

# Create the output directories for each class
output_dir_dme = "/content/drive/MyDrive/OCT-AND-EYE-FUNDUS-DATASET-main/fundus_DME"
output_dir_dr = "/content/drive/MyDrive/OCT-AND-EYE-FUNDUS-DATASET-main/fundus_DR"
output_dir_healthy = "/content/drive/MyDrive/OCT-AND-EYE-FUNDUS-DATASET-main/fundus_HEALTHY"

os.makedirs(output_dir_dme, exist_ok=True)
os.makedirs(output_dir_dr, exist_ok=True)
os.makedirs(output_dir_healthy, exist_ok=True)

# Iterate through the CSV rows and move images to corresponding class folders
for index, row in data.iterrows():
    image_id = row['Name']
    dme_label = row['DME']
    dr_label = row['DR']

    image_path = os.path.join(images_folder, image_id + ".jpg")

    if dme_label == 1:
        shutil.copy(image_path, os.path.join(output_dir_dme, image_id + ".jpg"))
    elif dme_label == 0 and dr_label == 1:
        shutil.copy(image_path, os.path.join(output_dir_dr, image_id + ".jpg"))
    elif dme_label == 0 and dr_label == 0:
        shutil.copy(image_path, os.path.join(output_dir_healthy, image_id + ".jpg"))


In [ ]:
import os
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from imblearn.over_sampling import RandomOverSampler

# Set the paths to the dataset folders
data_dir = "path_to_dataset_folder"
output_dir = "path_to_preprocessed_output_folder"

# Define the target image size for resizing
target_size = (256, 256)  # Adjust as per your requirements

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Resize and augment the images
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

# Load the images and labels
images = []
labels = []

for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)
    if os.path.isdir(class_dir):
        label = class_name
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image = cv2.imread(image_path)
            image = cv2.resize(image, target_size)
            images.append(image)
            labels.append(label)

# Convert images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Shuffle the dataset
images, labels = shuffle(images, labels, random_state=42)

# Perform oversampling to handle imbalanced classes
oversampler = RandomOverSampler(random_state=42)
images, labels = oversampler.fit_resample(images.reshape(-1, np.prod(target_size * 3)), labels)
images = images.reshape(-1, *target_size, 3)

# Save the preprocessed images
for i in range(len(images)):
    image_path = os.path.join(output_dir, f"image_{i}.jpg")
    cv2.imwrite(image_path, images[i])

# Save the labels to a text file
label_path = os.path.join(output_dir, "labels.txt")
np.savetxt(label_path, labels, fmt="%s")

print("Preprocessing complete.")
